In [ ]:
import pandas as pd
import PyPDF2
import requests
from io import BytesIO


df = pd.read_csv('appeal_document.csv')

def extract_text_and_sentences_with_numbers_from_url(url):
    try:
        if pd.notna(url):  # Check if the URL is not NaN
            response = requests.get(url)
            pdf_data = BytesIO(response.content)
            pdf_reader = PyPDF2.PdfFileReader(pdf_data)

            full_text = ""
            sentences_with_numbers = []

            for page_num in range(pdf_reader.numPages):
                page = pdf_reader.getPage(page_num)
                full_text += page.extractText()
                text = page.extractText()

                # Split the text into sentences using period (.) as the separator
                sentences = re.split(r'(?<=[.!?])\s+', text)

                # Check each sentence for the presence of numbers
                for sentence in sentences:
                    if any(char.isdigit() for char in sentence):
                        sentences_with_numbers.append(sentence)

            return full_text, sentences_with_numbers
        else:
            return None, None  # Return None for both columns when the URL is NaN
    except Exception as e:
        return str(e)

# Create a new column 'full_text' and 'sentences_with_numbers' containing full text and sentences with numbers
df[['full_text', 'sentences_with_numbers']] = df['document_url'].apply(extract_text_and_sentences_with_numbers_from_url)

#Saving dataframe in a csv
df.to_csv('appeal_document_with_pdf_text.csv',index=False)

